# 20 nm Origami Localization Precision Measurement

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import colorsys
import matplotlib.gridspec as gridspec

In [ ]:
# Define File Location
file = '/Users/abhinav/Library/CloudStorage/OneDrive-IndianInstituteofScience/Papers/zz_Msequences/Multiplexing/20250116/CrossTalk/R2_on_R10_avg3.hdf5' # <<< Set your file path here
parent_folder = '/Users/abhinav/Library/CloudStorage/OneDrive-IndianInstituteofScience/Papers/zz_Msequences/Multiplexing/20250116/CrossTalk/'
imager_name = 'R2'
imagers = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12']
hues = np.arange(0, 1, 1 / 12)
colors = [colorsys.hsv_to_rgb(_, 1, 1) for _ in hues]
colors = colors[-6:] + colors[:-6]
color_map = dict(zip(imagers, colors))

In [ ]:
# known physical spacing
spacing_nm = 20
pixel_nm = 130
spacing = spacing_nm / pixel_nm  # ~0.1538 units

# expected peak centers
x_centers = np.linspace(-1.5 * spacing, 1.5 * spacing, 4)
y_centers = np.linspace(-1.0 * spacing, 1.0 * spacing, 3)

In [ ]:
# Load Data
data = pd.read_hdf(file, key='locs')
x = data['x'].to_numpy()
y = data['y'].to_numpy()

# Center Data
x_centered = x - np.mean(x)
y_centered = y - np.mean(y)

In [ ]:
def multi_gaussian(x, *params):
    """
    Sum of N Gaussians.
    params = [amp1, mean1, sigma1, amp2, mean2, sigma2, ...]
    """
    n_gauss = len(params) // 3
    y = np.zeros_like(x, dtype=float)
    for i in range(n_gauss):
        amp, mean, sigma = params[3*i : 3*(i+1)]
        y += amp * np.exp(-(x - mean)**2 / (2 * sigma**2))
    return y

In [ ]:
# assuming your data is in arrays x_points, y_points
# and you want histogram projections
x_hist, x_edges = np.histogram(x_centered, bins=200)
x_centers_hist = 0.5 * (x_edges[:-1] + x_edges[1:])

y_hist, y_edges = np.histogram(y_centered, bins=200)
y_centers_hist = 0.5 * (y_edges[:-1] + y_edges[1:])

# --- X projection fit ---
p0_x = []
for xc in x_centers:
    p0_x += [x_hist.max(), xc, 0.02]   # amplitude, mean, sigma guess
popt_x, _ = curve_fit(multi_gaussian, x_centers_hist, x_hist, p0=p0_x)

# --- Y projection fit ---
p0_y = []
for yc in y_centers:
    p0_y += [y_hist.max(), yc, 0.02]
popt_y, _ = curve_fit(multi_gaussian, y_centers_hist, y_hist, p0=p0_y)

In [ ]:
# extract means and sigmas
x_means = [popt_x[i*3 + 1] for i in range(4)]
x_sigmas = [popt_x[i*3 + 2] for i in range(4)]
y_means = [popt_y[i*3 + 1] for i in range(3)]
y_sigmas = [popt_y[i*3 + 2] for i in range(3)]

x_means_nm = x_means * pixel_nm
y_means_nm = y_means * pixel_nm
x_sigmas_nm = np.array(x_sigmas) * pixel_nm
y_sigmas_nm = np.array(y_sigmas) * pixel_nm

# print("X centers:", x_means_nm)
print("X stds:", x_sigmas_nm)
print("X std mean:", np.mean(x_sigmas_nm))
# print("Y centers:", y_means_nm)
print("Y stds:", y_sigmas_nm)
print("Y std mean:", np.mean(y_sigmas_nm))
print("Overall std mean:", np.mean(np.concatenate([x_sigmas_nm, y_sigmas_nm])))

# --- Convert your pixel coordinates to nm ---
x_points_nm = x_centered * pixel_nm
y_points_nm = y_centered * pixel_nm

# Convert histogram bin centers to nm as well
x_centers_nm = x_centers_hist * pixel_nm
y_centers_nm = y_centers_hist * pixel_nm

# --- Create the figure layout ---
fig = plt.figure(figsize=(7,7))
gs = gridspec.GridSpec(4, 4, figure=fig, wspace=0.05, hspace=0.05)

# Main scatter plot
ax_main = fig.add_subplot(gs[1:4, 0:3])

# Top histogram (X projection)
ax_top = fig.add_subplot(gs[0, 0:3], sharex=ax_main)

# Right histogram (Y projection)
ax_right = fig.add_subplot(gs[1:4, 3], sharey=ax_main)

# --- Scatter plot (center) ---
ax_main.scatter(x_points_nm, y_points_nm, s=2, alpha=0.4, linewidths=0.1, color = color_map[imager_name], edgecolors='k')
ax_main.set_xlabel("X (nm)")
ax_main.set_ylabel("Y (nm)")
ax_main.set_aspect('equal')
ax_main.set_xlim(-60,60)
ax_main.set_ylim(-60,60)
# ax_main.grid(alpha=0.25, linestyle='--')

# --- X projection (top) ---
ax_top.plot(x_centers_nm, x_hist, color='gray', lw=2)
ax_top.plot(x_centers_nm, multi_gaussian(x_centers_hist, *popt_x), color = 'k', lw=0.8)
ax_top.set_ylabel("Counts")
ax_top.tick_params(axis="x", labelbottom=False) # hide x labels for top plot
# ax_top.plot(axis='off')

# --- Y projection (right) ---
# We plot horizontally, so x = counts, y = position (nm)
ax_right.plot(y_hist, y_centers_nm, color='gray', lw=2)
ax_right.plot(multi_gaussian(y_centers_hist, *popt_y), y_centers_nm, color='k', lw=0.8)
ax_right.set_xlabel("Counts")
ax_right.tick_params(axis="y", labelleft=False)  # hide y labels for right plot

# --- Set matching axis limits ---
# ax_main.set_xlim(x_centers_nm.min(), x_centers_nm.max())
# ax_main.set_ylim(y_centers_nm.min(), y_centers_nm.max())
ax_top.set_xlim(-60, 60)
ax_right.set_ylim(-60, 60)

# --- Adjust layout neatly ---
plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)

# --- Add Imager Name and S.D. at the top right of the figure ---
sd_value_nm = np.mean(np.concatenate([x_sigmas_nm, y_sigmas_nm]))  # example value, replace with your real SD

fig.text(
    0.95, 0.95, 
    f"Imager Name: {imager_name}\nS.D.: {sd_value_nm:.2f} nm",
    ha='right', va='top',
    fontsize=10,
    fontweight='medium'
)

output_folder = parent_folder
plt.savefig(f"{output_folder}/{imager_name}_scatter_plot_R10.svg", format = 'svg', bbox_inches='tight')
plt.show()